<a href="https://colab.research.google.com/github/z27833009/Deep_Learning_4_NLP_Homework/blob/main/Homework_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Colab Notebooks/hw4

MessageError: ignored

In [ ]:
!pip install torch
!pip install transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 11.9 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import GPT2Tokenizer, AutoTokenizer, GPT2LMHeadModel
from datasets import load_dataset
from collections import defaultdict

## **Task 1: Byte pair encoding**

The goal is to calculate byte pair encodings for a given dataset. For a recap of the concept we refer to lecture 9, slides 22.ff and internet search.

### Task 1.1
First we want to make sure we create a suffieciently large corpus. As dataset you import the dataset "wikitext-2-raw-v1" from huggingface. Make sure to use the 500 first entries of the training dataset and subsequently filter out empty  entries. Also print the first entry of the corpus.

In [ ]:
# Load WikiText-2 dataset "wikitext-2-raw-v1"
corpus = []
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

# Extract the first 500 entries from the training split
corpus = dataset["train"][:500]

# Filter out empty entries
corpus = [sentence for sentence in corpus["text"] if len(sentence) > 0]


Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

Dataset wikitext downloaded and prepared to /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
print(len(corpus))
print(corpus[0])

338
 = Valkyria Chronicles III = 



Note that the corpus should have a format like
corpus = [
    "This is the first sentence",
    "This is the second sentence",
    ..
]. For debugging during coding you can also consider working with a smaller corpus, e.g. only 5 sentences

### Task 1.2
Here, you loop through the corpus and count the word frequencies

In [ ]:
# Initialize a tokenizer from the transformers library
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Initialize a defaultdict to store word frequencies
word_freqs = defaultdict(int)
# this dictionary gets entries for word_freqs[word]

alphabet = []
vocab = [""]
splits = {}
vocab_size = 50
merges = {}

# We loop through to corups to calculate word frequencies
for text in corpus:
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in words_with_offsets]

    for word in new_words:
        word_freqs[word] += 1

print(word_freqs)


defaultdict(<class 'int'>, {'Ġ=': 354, 'ĠValkyria': 54, 'ĠChronicles': 39, 'ĠIII': 15, 'ĠĊ': 338, 'ĠSenjÅį': 5, 'Ġno': 20, 'Ġ3': 29, 'Ġ:': 46, 'ĠUnrecorded': 1, 'Ġ(': 130, 'ĠJapanese': 6, 'ĠæĪ¦åł´ãģ®ãĥ´ãĤ¡ãĥ«ãĤŃãĥ¥ãĥªãĤ¢': 4, '3': 3, 'Ġ,': 1053, 'Ġlit': 4, 'Ġ.': 815, 'Ġof': 585, 'Ġthe': 1349, 'ĠBattlefield': 4, 'Ġ)': 130, 'Ġcommonly': 1, 'Ġreferred': 4, 'Ġto': 452, 'Ġas': 117, 'Ġoutside': 7, 'ĠJapan': 4, 'Ġis': 74, 'Ġa': 350, 'Ġtactical': 3, 'Ġrole': 6, 'Ġ@-@': 209, 'Ġplaying': 5, 'Ġvideo': 7, 'Ġgame': 74, 'Ġdeveloped': 4, 'Ġby': 131, 'ĠSega': 5, 'Ġand': 546, 'ĠMedia': 4, '.': 100, 'Vision': 4, 'Ġfor': 188, 'ĠPlayStation': 9, 'ĠPortable': 3, 'ĠReleased': 1, 'Ġin': 394, 'ĠJanuary': 11, 'Ġ2011': 17, 'Ġit': 66, 'Ġthird': 14, 'Ġseries': 20, 'ĠEmploying': 1, 'Ġsame': 15, 'Ġfusion': 1, 'Ġreal': 6, 'Ġtime': 31, 'Ġgameplay': 10, 'Ġits': 37, 'Ġpredecessors': 2, 'Ġstory': 14, 'Ġruns': 1, 'Ġparallel': 1, 'Ġfirst': 49, 'Ġfollows': 2, 'Ġ"': 182, 'ĠNameless': 12, 'Ġpenal': 2, 'Ġmilitary': 13, 'Ġunit

The following block creates the alphabet and initial vocabulary

In [ ]:
for word in word_freqs.keys():
    for letter in word:
        if letter not in alphabet:
            alphabet.append(letter)
alphabet.sort()

print("alphabet",alphabet)
vocab = ["<|endoftext|>"] + alphabet.copy()
splits = {word: [c for c in word] for word in word_freqs.keys()}

alphabet ['!', '"', '$', '%', '&', "'", '(', ')', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '~', '¡', '¢', '£', '¤', '¥', '¦', '©', 'ª', '«', '®', '¯', '°', '±', '²', '³', '´', 'µ', '¹', '¼', '½', '¿', 'Â', 'Ã', 'Å', 'â', 'ã', 'ä', 'å', 'æ', 'ç', 'è', 'é', 'ï', 'Ċ', 'Ġ', 'Ģ', 'ģ', 'Ĥ', 'ĥ', 'Ħ', 'ħ', 'Ĩ', 'Ī', 'ī', 'Ĭ', 'ĭ', 'Į', 'į', 'ı', 'Ĳ', 'ĳ', 'Ĵ', 'ĵ', 'Ķ', 'Ĺ', 'ĺ', 'Ļ', 'Ľ', 'Ł', 'ł', 'Ń']


In [ ]:
print(splits)
print(vocab)
print(len(vocab))

{'Ġ=': ['Ġ', '='], 'ĠValkyria': ['Ġ', 'V', 'a', 'l', 'k', 'y', 'r', 'i', 'a'], 'ĠChronicles': ['Ġ', 'C', 'h', 'r', 'o', 'n', 'i', 'c', 'l', 'e', 's'], 'ĠIII': ['Ġ', 'I', 'I', 'I'], 'ĠĊ': ['Ġ', 'Ċ'], 'ĠSenjÅį': ['Ġ', 'S', 'e', 'n', 'j', 'Å', 'į'], 'Ġno': ['Ġ', 'n', 'o'], 'Ġ3': ['Ġ', '3'], 'Ġ:': ['Ġ', ':'], 'ĠUnrecorded': ['Ġ', 'U', 'n', 'r', 'e', 'c', 'o', 'r', 'd', 'e', 'd'], 'Ġ(': ['Ġ', '('], 'ĠJapanese': ['Ġ', 'J', 'a', 'p', 'a', 'n', 'e', 's', 'e'], 'ĠæĪ¦åł´ãģ®ãĥ´ãĤ¡ãĥ«ãĤŃãĥ¥ãĥªãĤ¢': ['Ġ', 'æ', 'Ī', '¦', 'å', 'ł', '´', 'ã', 'ģ', '®', 'ã', 'ĥ', '´', 'ã', 'Ĥ', '¡', 'ã', 'ĥ', '«', 'ã', 'Ĥ', 'Ń', 'ã', 'ĥ', '¥', 'ã', 'ĥ', 'ª', 'ã', 'Ĥ', '¢'], '3': ['3'], 'Ġ,': ['Ġ', ','], 'Ġlit': ['Ġ', 'l', 'i', 't'], 'Ġ.': ['Ġ', '.'], 'Ġof': ['Ġ', 'o', 'f'], 'Ġthe': ['Ġ', 't', 'h', 'e'], 'ĠBattlefield': ['Ġ', 'B', 'a', 't', 't', 'l', 'e', 'f', 'i', 'e', 'l', 'd'], 'Ġ)': ['Ġ', ')'], 'Ġcommonly': ['Ġ', 'c', 'o', 'm', 'm', 'o', 'n', 'l', 'y'], 'Ġreferred': ['Ġ', 'r', 'e', 'f', 'e', 'r', 'r', 'e', 'd'], 'Ġt

### Task 1.3
Define a function to compute the frequency of each pair of characters

In [ ]:
# Define a function to compute the frequency of each pair of characters
def compute_pair_freqs(splits):
    pair_freqs = defaultdict(int)
    for value in splits.values():
      for i in range(len(value)-1):
        pair = (value[i] , value[i+1])
        pair_freqs[pair] += 1


    return pair_freqs

# Initialize pair_freqs by calling the compute_pair_freqs function
pair_freqs = compute_pair_freqs(splits)

In [ ]:
print(pair_freqs)
print(len(pair_freqs))

defaultdict(<class 'int'>, {('Ġ', '='): 1, ('Ġ', 'V'): 29, ('V', 'a'): 10, ('a', 'l'): 311, ('l', 'k'): 9, ('k', 'y'): 5, ('y', 'r'): 6, ('r', 'i'): 235, ('i', 'a'): 102, ('Ġ', 'C'): 115, ('C', 'h'): 27, ('h', 'r'): 18, ('r', 'o'): 183, ('o', 'n'): 467, ('n', 'i'): 86, ('i', 'c'): 191, ('c', 'l'): 43, ('l', 'e'): 244, ('e', 's'): 377, ('Ġ', 'I'): 33, ('I', 'I'): 7, ('Ġ', 'Ċ'): 1, ('Ġ', 'S'): 127, ('S', 'e'): 19, ('e', 'n'): 365, ('n', 'j'): 9, ('j', 'Å'): 1, ('Å', 'į'): 3, ('Ġ', 'n'): 74, ('n', 'o'): 75, ('Ġ', '3'): 14, ('Ġ', ':'): 1, ('Ġ', 'U'): 16, ('U', 'n'): 7, ('n', 'r'): 6, ('r', 'e'): 420, ('e', 'c'): 138, ('c', 'o'): 208, ('o', 'r'): 305, ('r', 'd'): 69, ('d', 'e'): 252, ('e', 'd'): 522, ('Ġ', '('): 1, ('Ġ', 'J'): 30, ('J', 'a'): 12, ('a', 'p'): 77, ('p', 'a'): 98, ('a', 'n'): 322, ('n', 'e'): 188, ('s', 'e'): 241, ('Ġ', 'æ'): 1, ('æ', 'Ī'): 2, ('Ī', '¦'): 2, ('¦', 'å'): 1, ('å', 'ł'): 1, ('ł', '´'): 1, ('´', 'ã'): 2, ('ã', 'ģ'): 20, ('ģ', '®'): 6, ('®', 'ã'): 2, ('ã', 'ĥ'): 4,

Here we find the pair of characters that appears most frequently together

In [ ]:
for i, key in enumerate(pair_freqs.keys()):
    print(f"{key}: {pair_freqs[key]}")
    if i >= 5:
        break

best_pair = ""
max_freq = None

for pair, freq in pair_freqs.items():
    if max_freq is None or max_freq < freq:
        best_pair = pair
        max_freq = freq

print('best_pair: ',best_pair, max_freq)

('Ġ', '='): 1
('Ġ', 'V'): 29
('V', 'a'): 10
('a', 'l'): 311
('l', 'k'): 9
('k', 'y'): 5
best_pair:  ('i', 'n') 582


### Task 1.4
Define a function to merge the most frequent pair of characters

In [ ]:

# Define a function to merge the most frequent pair of characters
def merge_pair(a, b, splits):
    for word in word_freqs:
        split = splits[word]
        if len(split) == 1:
            continue
        merged_word = []
        i = 0
        while i < len(split)-1:
          if split[i]==a and split[i+1]==b:
            merged_word.append(a+b)
            i += 2
          else:
            merged_word.append(split[i])
            i += 1
        if i == len(split)-1:
          merged_word.append(split[-1])

        splits[word] = merged_word

    return splits


In [ ]:
print(merge_pair('i','n',splits))

{'Ġ=': ['Ġ', '='], 'ĠValkyria': ['Ġ', 'V', 'a', 'l', 'k', 'y', 'r', 'i', 'a'], 'ĠChronicles': ['Ġ', 'C', 'h', 'r', 'o', 'n', 'i', 'c', 'l', 'e', 's'], 'ĠIII': ['Ġ', 'I', 'I', 'I'], 'ĠĊ': ['Ġ', 'Ċ'], 'ĠSenjÅį': ['Ġ', 'S', 'e', 'n', 'j', 'Å', 'į'], 'Ġno': ['Ġ', 'n', 'o'], 'Ġ3': ['Ġ', '3'], 'Ġ:': ['Ġ', ':'], 'ĠUnrecorded': ['Ġ', 'U', 'n', 'r', 'e', 'c', 'o', 'r', 'd', 'e', 'd'], 'Ġ(': ['Ġ', '('], 'ĠJapanese': ['Ġ', 'J', 'a', 'p', 'a', 'n', 'e', 's', 'e'], 'ĠæĪ¦åł´ãģ®ãĥ´ãĤ¡ãĥ«ãĤŃãĥ¥ãĥªãĤ¢': ['Ġ', 'æ', 'Ī', '¦', 'å', 'ł', '´', 'ã', 'ģ', '®', 'ã', 'ĥ', '´', 'ã', 'Ĥ', '¡', 'ã', 'ĥ', '«', 'ã', 'Ĥ', 'Ń', 'ã', 'ĥ', '¥', 'ã', 'ĥ', 'ª', 'ã', 'Ĥ', '¢'], '3': ['3'], 'Ġ,': ['Ġ', ','], 'Ġlit': ['Ġ', 'l', 'i', 't'], 'Ġ.': ['Ġ', '.'], 'Ġof': ['Ġ', 'o', 'f'], 'Ġthe': ['Ġ', 't', 'h', 'e'], 'ĠBattlefield': ['Ġ', 'B', 'a', 't', 't', 'l', 'e', 'f', 'i', 'e', 'l', 'd'], 'Ġ)': ['Ġ', ')'], 'Ġcommonly': ['Ġ', 'c', 'o', 'm', 'm', 'o', 'n', 'l', 'y'], 'Ġreferred': ['Ġ', 'r', 'e', 'f', 'e', 'r', 'r', 'e', 'd'], 'Ġt

### Task 1.5

Iterate this process until you reach a predefined vocabulary size

In [ ]:
def comm_pair(pair_freqs):
  best_pair = ""
  max_freq = None

  for pair, freq in pair_freqs.items():
    if max_freq is None or max_freq < freq:
        best_pair = pair
        max_freq = freq
  return best_pair,max_freq

In [ ]:
vocab_size = 500
# YOUR CODE HERE to iterate this process until you reach a predefined vocabulary size
while len(vocab) < vocab_size:
    # Compute pair frequencies
    pair_freqs = compute_pair_freqs(splits)

    # Find the most frequent pair
    best_pair = ""
    max_freq = None

    for pair, freq in pair_freqs.items():
      if max_freq is None or max_freq < freq:
        best_pair = pair
        max_freq = freq

    # Stop if no pair is found
    if max_freq is None:
        break

    # Merge the most frequent pair
    merges[best_pair] = max_freq
    splits = merge_pair(best_pair[0], best_pair[1], splits)

    del pair_freqs[best_pair]

    # Update the vocabulary
    vocab.append(best_pair[0] + best_pair[1])



print("Vocabulary:", vocab)
print("Vocabulary len:", len(vocab))



Vocabulary: ['<|endoftext|>', '!', '"', '$', '%', '&', "'", '(', ')', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '~', '¡', '¢', '£', '¤', '¥', '¦', '©', 'ª', '«', '®', '¯', '°', '±', '²', '³', '´', 'µ', '¹', '¼', '½', '¿', 'Â', 'Ã', 'Å', 'â', 'ã', 'ä', 'å', 'æ', 'ç', 'è', 'é', 'ï', 'Ċ', 'Ġ', 'Ģ', 'ģ', 'Ĥ', 'ĥ', 'Ħ', 'ħ', 'Ĩ', 'Ī', 'ī', 'Ĭ', 'ĭ', 'Į', 'į', 'ı', 'Ĳ', 'ĳ', 'Ĵ', 'ĵ', 'Ķ', 'Ĺ', 'ĺ', 'Ļ', 'Ľ', 'Ł', 'ł', 'Ń', 'in', 'er', 'ed', 'on', 'es', 'en', 'Ġs', 'at', 'an', 'al', 'or', 'ar', 'ing', 'Ġc', 'Ġp', 're', 'ti', 'is', 'Ġd', 'el', 'Ġm', 'Ġf', 'ic', 'it', 'Ġt', 'ou', 'le', 'Ġa', 'ro', 'ĠS', 'ion', 'Ġb', 'ent', 'ĠC', 'Ġre', 'Ġ1', 'il', 'om', 'as', 'ac', 'ec', 'Ġw',

In [ ]:
print(merges)

{('e', 'r'): 525, ('e', 'd'): 522, ('o', 'n'): 467, ('e', 's'): 377, ('e', 'n'): 365, ('Ġ', 's'): 362, ('a', 't'): 329, ('a', 'n'): 322, ('a', 'l'): 311, ('o', 'r'): 305, ('a', 'r'): 296, ('in', 'g'): 287, ('Ġ', 'c'): 283, ('Ġ', 'p'): 269, ('r', 'e'): 229, ('t', 'i'): 221, ('i', 's'): 196, ('Ġ', 'd'): 186, ('e', 'l'): 168, ('Ġ', 'm'): 162, ('Ġ', 'f'): 155, ('i', 'c'): 154, ('i', 't'): 150, ('Ġ', 't'): 150, ('o', 'u'): 150, ('l', 'e'): 133, ('Ġ', 'a'): 133, ('r', 'o'): 131, ('Ġ', 'S'): 127, ('i', 'on'): 125, ('Ġ', 'b'): 123, ('en', 't'): 116, ('Ġ', 'C'): 115, ('Ġ', 're'): 115, ('Ġ', '1'): 115, ('i', 'l'): 114, ('o', 'm'): 113, ('a', 's'): 108, ('a', 'c'): 107, ('e', 'c'): 103, ('Ġ', 'w'): 100, ('Ġ', 'A'): 97, ('l', 'o'): 93, ('s', 't'): 93, ('l', 'y'): 92, ('Ġ', 'e'): 91, ('Ġ', 'M'): 89, ('e', 't'): 89, ('Ġ', 'in'): 88, ('Ġ', 'P'): 87, ('Ġ', 'h'): 87, ('u', 'r'): 85, ('er', 's'): 85, ('r', 'a'): 82, ('r', 'i'): 80, ('Ġ', 'g'): 79, ('at', 'ion'): 79, ('t', 'h'): 79, ('Ġ', 'B'): 75, ('Ġ',

In [ ]:
print(splits)

{'Ġ=': ['Ġ', '='], 'ĠValkyria': ['ĠV', 'al', 'k', 'y', 'ri', 'a'], 'ĠChronicles': ['ĠCh', 'ron', 'ic', 'les'], 'ĠIII': ['ĠI', 'I', 'I'], 'ĠĊ': ['Ġ', 'Ċ'], 'ĠSenjÅį': ['ĠS', 'en', 'j', 'Å', 'į'], 'Ġno': ['Ġn', 'o'], 'Ġ3': ['Ġ3'], 'Ġ:': ['Ġ', ':'], 'ĠUnrecorded': ['ĠU', 'n', 'rec', 'ord', 'ed'], 'Ġ(': ['Ġ', '('], 'ĠJapanese': ['ĠJ', 'ap', 'an', 'es', 'e'], 'ĠæĪ¦åł´ãģ®ãĥ´ãĤ¡ãĥ«ãĤŃãĥ¥ãĥªãĤ¢': ['Ġ', 'æ', 'Ī', '¦', 'å', 'ł', '´', 'ãģ', '®', 'ã', 'ĥ', '´', 'ã', 'Ĥ', '¡', 'ã', 'ĥ', '«', 'ã', 'Ĥ', 'Ń', 'ã', 'ĥ', '¥', 'ã', 'ĥ', 'ª', 'ã', 'Ĥ', '¢'], '3': ['3'], 'Ġ,': ['Ġ', ','], 'Ġlit': ['Ġl', 'it'], 'Ġ.': ['Ġ', '.'], 'Ġof': ['Ġo', 'f'], 'Ġthe': ['Ġth', 'e'], 'ĠBattlefield': ['ĠB', 'att', 'le', 'f', 'i', 'el', 'd'], 'Ġ)': ['Ġ', ')'], 'Ġcommonly': ['Ġcomm', 'on', 'ly'], 'Ġreferred': ['Ġre', 'f', 'er', 'red'], 'Ġto': ['Ġt', 'o'], 'Ġas': ['Ġa', 's'], 'Ġoutside': ['Ġ', 'ou', 'ts', 'ide'], 'ĠJapan': ['ĠJ', 'ap', 'an'], 'Ġis': ['Ġ', 'is'], 'Ġa': ['Ġa'], 'Ġtactical': ['Ġt', 'ac', 'tic', 'al'], 'Ġrole': 

We provide the tokenize function to you

In [ ]:
def tokenize(text):
    pre_tokenize_result = tokenizer._tokenizer.pre_tokenizer.pre_tokenize_str(text)
    pre_tokenized_text = [word for word, offset in pre_tokenize_result]
    splits = [[l for l in word] for word in pre_tokenized_text]
    for pair, merge in merges.items():
        for idx, split in enumerate(splits):
            i = 0
            while i < len(split) - 1:
                if split[i] == pair[0] and split[i + 1] == pair[1]:
                    split = split[:i] + [merge] + split[i + 2 :]
                else:
                    i += 1
            splits[idx] = split

    return sum(splits, [])
tokenize("Darmstadt holds the official title City of Science (German: Wissenschaftsstadt) as it is a major centre of scientific institutions, universities, and high-technology companies.")

['D',
 296,
 'm',
 93,
 62,
 't',
 87,
 68,
 'd',
 's',
 150,
 'h',
 'e',
 57,
 44,
 154,
 'i',
 311,
 'Ġ',
 221,
 't',
 133,
 115,
 150,
 'y',
 57,
 'f',
 127,
 24,
 365,
 51,
 'Ġ',
 '(',
 'G',
 525,
 'm',
 322,
 ':',
 52,
 196,
 's',
 365,
 's',
 63,
 'a',
 'f',
 55,
 93,
 62,
 't',
 ')',
 133,
 's',
 'Ġ',
 150,
 'Ġ',
 196,
 133,
 162,
 'a',
 'j',
 305,
 283,
 365,
 't',
 229,
 57,
 'f',
 362,
 24,
 365,
 221,
 'f',
 154,
 'Ġ',
 582,
 's',
 221,
 't',
 'u',
 221,
 467,
 's',
 ',',
 'Ġ',
 74,
 58,
 525,
 's',
 'i',
 221,
 377,
 ',',
 'Ġ',
 322,
 'd',
 87,
 56,
 'h',
 '-',
 't',
 103,
 'h',
 'n',
 'o',
 93,
 'g',
 'y',
 283,
 113,
 'p',
 322,
 'i',
 377,
 '.']

### Task 1.6
Now, repeat the byte pair encoding for vocab sizes of 1000 and 5000. What do you notice?

In [ ]:
# Use this chunk for the answer
vocab_size = 1000
# YOUR CODE HERE to iterate this process until you reach a predefined vocabulary size
while len(vocab) < vocab_size:
    # Compute pair frequencies
    pair_freqs = compute_pair_freqs(splits)

    # Find the most frequent pair
    best_pair = ""
    max_freq = None

    for pair, freq in pair_freqs.items():
      if max_freq is None or max_freq < freq:
        best_pair = pair
        max_freq = freq

    # Stop if no pair is found
    if max_freq is None:
        break

    # Merge the most frequent pair
    merges[best_pair] = max_freq
    splits = merge_pair(best_pair[0], best_pair[1], splits)

    del pair_freqs[best_pair]

    # Update the vocabulary
    vocab.append(best_pair[0] + best_pair[1])


print("Merges:", merges)
print("Vocabulary:", vocab)
print("Vocabulary len:", len(vocab))



Merges: {('i', 'n'): 582, ('e', 'r'): 525, ('e', 'd'): 522, ('o', 'n'): 467, ('e', 's'): 377, ('e', 'n'): 365, ('Ġ', 's'): 362, ('a', 't'): 329, ('a', 'n'): 322, ('a', 'l'): 311, ('o', 'r'): 305, ('a', 'r'): 296, ('in', 'g'): 287, ('Ġ', 'c'): 283, ('Ġ', 'p'): 269, ('r', 'e'): 229, ('t', 'i'): 221, ('i', 's'): 196, ('Ġ', 'd'): 186, ('e', 'l'): 168, ('Ġ', 'm'): 162, ('Ġ', 'f'): 155, ('i', 'c'): 154, ('i', 't'): 150, ('Ġ', 't'): 150, ('o', 'u'): 150, ('l', 'e'): 133, ('Ġ', 'a'): 133, ('r', 'o'): 131, ('Ġ', 'S'): 127, ('i', 'on'): 125, ('Ġ', 'b'): 123, ('en', 't'): 116, ('Ġ', 'C'): 115, ('Ġ', 're'): 115, ('Ġ', '1'): 115, ('i', 'l'): 114, ('o', 'm'): 113, ('a', 's'): 108, ('a', 'c'): 107, ('e', 'c'): 103, ('Ġ', 'w'): 100, ('Ġ', 'A'): 97, ('l', 'o'): 93, ('s', 't'): 93, ('l', 'y'): 92, ('Ġ', 'e'): 91, ('Ġ', 'M'): 89, ('e', 't'): 89, ('Ġ', 'in'): 88, ('Ġ', 'P'): 87, ('Ġ', 'h'): 87, ('u', 'r'): 85, ('er', 's'): 85, ('r', 'a'): 82, ('r', 'i'): 80, ('Ġ', 'g'): 79, ('at', 'ion'): 79, ('t', 'h'): 

In [ ]:
vocab_size = 5000
# YOUR CODE HERE to iterate this process until you reach a predefined vocabulary size
while len(vocab) < vocab_size:
    # Compute pair frequencies
    pair_freqs = compute_pair_freqs(splits)

    # Find the most frequent pair
    best_pair = ""
    max_freq = None

    for pair, freq in pair_freqs.items():
      if max_freq is None or max_freq < freq:
        best_pair = pair
        max_freq = freq

    # Stop if no pair is found
    if max_freq is None:
        break

    # Merge the most frequent pair
    merges[best_pair] = max_freq
    splits = merge_pair(best_pair[0], best_pair[1], splits)

    del pair_freqs[best_pair]

    # Update the vocabulary
    vocab.append(best_pair[0] + best_pair[1])


print("Merges:", merges)
print("Vocabulary:", vocab)
print("Vocabulary len:", len(vocab))


Merges: {('i', 'n'): 582, ('e', 'r'): 525, ('e', 'd'): 522, ('o', 'n'): 467, ('e', 's'): 377, ('e', 'n'): 365, ('Ġ', 's'): 362, ('a', 't'): 329, ('a', 'n'): 322, ('a', 'l'): 311, ('o', 'r'): 305, ('a', 'r'): 296, ('in', 'g'): 287, ('Ġ', 'c'): 283, ('Ġ', 'p'): 269, ('r', 'e'): 229, ('t', 'i'): 221, ('i', 's'): 196, ('Ġ', 'd'): 186, ('e', 'l'): 168, ('Ġ', 'm'): 162, ('Ġ', 'f'): 155, ('i', 'c'): 154, ('i', 't'): 150, ('Ġ', 't'): 150, ('o', 'u'): 150, ('l', 'e'): 133, ('Ġ', 'a'): 133, ('r', 'o'): 131, ('Ġ', 'S'): 127, ('i', 'on'): 125, ('Ġ', 'b'): 123, ('en', 't'): 116, ('Ġ', 'C'): 115, ('Ġ', 're'): 115, ('Ġ', '1'): 115, ('i', 'l'): 114, ('o', 'm'): 113, ('a', 's'): 108, ('a', 'c'): 107, ('e', 'c'): 103, ('Ġ', 'w'): 100, ('Ġ', 'A'): 97, ('l', 'o'): 93, ('s', 't'): 93, ('l', 'y'): 92, ('Ġ', 'e'): 91, ('Ġ', 'M'): 89, ('e', 't'): 89, ('Ġ', 'in'): 88, ('Ġ', 'P'): 87, ('Ġ', 'h'): 87, ('u', 'r'): 85, ('er', 's'): 85, ('r', 'a'): 82, ('r', 'i'): 80, ('Ġ', 'g'): 79, ('at', 'ion'): 79, ('t', 'h'): 

# **Task 2: Getting to Know Foundation Models**

In this task, we want to load two different versions of GPT-2 and compare their performance.

### Task 2.1

After we imported transformers from Huggingface, your first task is to initialize the GPT2LMHead Model. This model should be a gpt2 model from huggingface under the variable model. Also define the GPT2 tokenizer.

While we have seen the transformers in the lectures, you should print a single GPT2Block to the console for the 10th layer. Then, count the parameters of the model.

In [ ]:
# Initialize GPT2 model
model_small = GPT2LMHeadModel.from_pretrained('gpt2')

# Initialize GPT2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Print a GPT2Block in the 10th layer
print(model_small.transformer.h[9])

def count_parameters(model):
     return sum(p.numel() for p in model_small.parameters() if p.requires_grad)

# GPT2_small parameter trainable parameters:
print(f"The model has {count_parameters(model_small):,} trainable parameters")

GPT2Block(
  (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (attn): GPT2Attention(
    (c_attn): Conv1D()
    (c_proj): Conv1D()
    (attn_dropout): Dropout(p=0.1, inplace=False)
    (resid_dropout): Dropout(p=0.1, inplace=False)
  )
  (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (mlp): GPT2MLP(
    (c_fc): Conv1D()
    (c_proj): Conv1D()
    (act): NewGELUActivation()
    (dropout): Dropout(p=0.1, inplace=False)
  )
)
The model has 124,439,808 trainable parameters


### Task 2.2
Because we want to compare two versions of GPT, you now also load the gpt2-large from huggingface as model. As evaluation dataset you import the dataset "wikitext-2-raw-v1" from huggingface. (like in Task1) Make sure to use the 200 first entries of the test dataset and print an entry. Also print the parameters of the gpt2-large model

In [ ]:
# Load the gpt2-large model from huggingface
model_large = GPT2LMHeadModel.from_pretrained('gpt2-large')


In [ ]:
# Load the dataset
dataset = load_dataset('wikitext', 'wikitext-2-raw-v1', split='test[:200]')

# Print any single entry
print(dataset[100])
print(type(dataset[100]))

# Print Trainable parameters of huggingface gpt2-large
print(f"The model has {count_parameters(model_large):,} trainable parameters")


{'text': ' Du Fu \'s popularity grew to such an extent that it is as hard to measure his influence as that of Shakespeare in England : it was hard for any Chinese poet not to be influenced by him . While there was never another Du Fu , individual poets followed in the traditions of specific aspects of his work : Bai Juyi \'s concern for the poor , Lu You \'s patriotism , and Mei Yaochen \'s reflections on the quotidian are a few examples . More broadly , Du Fu \'s work in transforming the lǜshi from mere word play into " a vehicle for serious poetic utterance " set the stage for every subsequent writer in the genre . \n'}
<class 'dict'>
The model has 124,439,808 trainable parameters


In [ ]:
new_dataset = [entry for entry in dataset if entry['text'] != '']



### Task 2.3
As a next step, we define an evaluation metric to calculate predictions. Remember the perplexity funciton from the lecture. The function should take a model, tokenizer and a text and return the perplexity score for the given text.

Hint: you should encode the text, generate outputs for inputs and corresponding labels and then calculate perplexity.

In [ ]:
import torch
from torch.nn.functional import softmax

def perplexity(model, tokenizer, text):
    if len(text) == 0:
        return float('inf')

    encoded_input = tokenizer.encode_plus(text, return_tensors='pt')

    with torch.no_grad():
        input_ids = encoded_input["input_ids"].to(model.device)
        attention_mask = encoded_input["attention_mask"].to(model.device)

        logits = model(input_ids, attention_mask=attention_mask).logits
        labels = input_ids[:, 1:].contiguous().view(-1)

        num_tokens = labels.size(0)
        log_probs = softmax(logits, dim=-1).view(-1, model.config.vocab_size)

        perplexity = torch.exp(-torch.log(log_probs.gather(dim=-1, index=labels.unsqueeze(1))).sum() / num_tokens)

    return perplexity.item()

In [ ]:
import numpy as np

perplexities_small = []
perplexities_large = []

# Loop over the dataset and calculate perplexities
for example in new_dataset:
    text = example["text"]

    perplexity_small = perplexity(model_small, tokenizer, text)
    perplexities_small.append(perplexity_small)

    perplexity_large = perplexity(model_large, tokenizer, text)
    perplexities_large.append(perplexity_large)

# Calculate average perplexities
avg_perplexity_small = np.mean(perplexities_small)
avg_perplexity_large = np.mean(perplexities_large)

# Print average perplexities
print(f"Average Perplexity for gpt2-small: {avg_perplexity_small}")
print(f"Average Perplexity for gpt2-large: {avg_perplexity_large}")


Average Perplexity for gpt2-small: 509.78574769727646
Average Perplexity for gpt2-large: 433.09409140002344


What do you notice? Please explain.

### Task 2.4 Masking attention
What is an attention mask? When and why is it usually used?

Your answer here:

Your task is now to run a prompt in regular inference, but with an attention mask. You can reuse the gpt2_small model

In [ ]:
text_prompt = "The sun was setting behind"

# Step 1: Implement attention masking
def apply_attention_mask(input_ids, mask_start_idx, mask_end_idx):
    attention_mask = torch.ones_like(input_ids)  # Initialize attention mask with all ones
    attention_mask[:, mask_start_idx:mask_end_idx+1] = 0  # Set the region to mask as zeros in the attention mask
    return attention_mask


# Step 2: Apply attention masks
mask_start = 4  # Start index of the region to mask
mask_end = 6  # End index of the region to mask
input_ids = tokenizer.encode(text_prompt, return_tensors='pt')
attention_mask = apply_attention_mask(input_ids, mask_start, mask_end)

# Step 3: Generate output using modified attention mechanism
with torch.no_grad():
    outputs = model_small.generate(input_ids, attention_mask=attention_mask)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Step 4: Compare and analyze the generated outputs
print("Text Prompt:", text_prompt)
print("Generated Text:", generated_text)



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Text Prompt: The sun was setting behind
Generated Text: The sun was setting behind the morning of the first day of the month, and the sun was setting


In [ ]:
# Step 5: Run the same prompt without attention mask
with torch.no_grad():
    outputs = model_small.generate(input_ids)
    generated_text_no_mask = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Step 6: Compare and analyze the generated outputs
print("Text Prompt:", text_prompt)
print("Generated Text (with attention mask):", generated_text)
print("Generated Text (without attention mask):", generated_text_no_mask)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Text Prompt: The sun was setting behind
Generated Text (with attention mask): The sun was setting behind the morning of the first day of the month, and the sun was setting
Generated Text (without attention mask): The sun was setting behind the clouds, and the wind was blowing in the direction of the sun.


Step 6: Compare the two outputs and describe it in one to two sentences

Your answer here: